In [1]:
import pandas as pd, urllib.request
import requests, os
import bs4
from urllib.parse import quote
from pivottablejs import pivot_ui

In [9]:
#go to link, check status code
url = "https://web.mit.edu/airlinedata/www/Employees&Productivity.html"
response = requests.get(url)
print(response.status_code)
# get a response object from the URL and use .get to get the HTML content
response = urllib.request.urlopen('https://web.mit.edu/airlinedata/www/Employees&Productivity.html')
response
response.status
res = requests.get('https://web.mit.edu/airlinedata/www/Employees&Productivity.html')
AirlineSoup = bs4.BeautifulSoup(res.text, "lxml")
#Isolate the href links that do not end in .xls
link_list = AirlineSoup.find_all('a', href=lambda href: href and not href.endswith('.xls'))
airline_table = link_list[30:-1]
airline_table
# Initialize an empty DataFrame for the combined results
passenger_employee_ratio_2 = pd.DataFrame()

# Loop through each item in airline_table
for airline_info in airline_table:
    url = airline_info['href']
    # Encode the URL to handle spaces and other special characters
    encoded_url = quote(url, safe="/:")
    # Concatenate the base URL with the encoded relative URL
    full_url = 'https://web.mit.edu/airlinedata/www/' + encoded_url
    # Use pd.read_html() with the encoded URL
    df_list = pd.read_html(full_url)
    if df_list:  # Check if the list is not empty
        df = df_list[0]
        # Get the airline name
        Airline = df[0][1]
        # Transpose the dataframe  
        df = df.transpose()
        # Isolate the 43rd through 49th columns and show all rows
        df2 = df.iloc[:, 43:49]
        df1 = df.iloc[:, 2]
        # Combine the two dataframes
        df3 = pd.concat([df1, df2], axis=1)
        # Set the first row as the column headers
        df3.columns = df3.iloc[0]
        df3 = df3.drop(df3.index[0])
        # Change first header to Year and change to int64 to handle NaN values
        df3.columns.values[0] = 'Year'
        df3['Year'] = df3['Year'].astype('Int64')
        # Add airline column that duplicates the airline name
        df3['Airline'] = Airline
        # Only keep years from 2009 to 2019
        df3 = df3[(df3['Year'] >= 2009) & (df3['Year'] <= 2019)]
        # Append the processed DataFrame to the cumulative DataFrame
        passenger_employee_ratio_2 = pd.concat([passenger_employee_ratio_2, df3], ignore_index=True)

airline_list = ['American Airlines', 'Delta Airlines', 'United Airlines', 'Southwest Airlines', 'Jetblue Airways', 'Frontier Airlines', 'Allegiant Air', 'Alaska Airlines', 'Hawaian Airlines', 'Spirit Airlines']
#only keep rows with the required airline values
passenger_employee_ratio_2 = passenger_employee_ratio_2[passenger_employee_ratio_2['Airline'].isin(airline_list)]
#in the airlines column remove all characters after the first space
passenger_employee_ratio_2['Airline'] = passenger_employee_ratio_2['Airline'].str.split(' ').str[0]
#uppercase the column names
passenger_employee_ratio_2.columns = passenger_employee_ratio_2.columns.str.upper()
#change airline column to CARRIER
passenger_employee_ratio_2 = passenger_employee_ratio_2.rename(columns={'AIRLINE': 'CARRIER'})
#create csv file of passenger_employee_ratio_2 and save it to the current working directory. index is carrier
passenger_employee_ratio_2.to_csv('passenger_employee_ratio_cleaned.csv', index='CARRIER')

200


In [9]:
pivot_ui(passenger_employee_ratio_2)